import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

In [2]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
rate=480
cls_num=2
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Ham(i)
os.chdir(cwd)
clses_lens=np.array([])
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
print('\nshape ecg=',np.shape(ecg))

first_train_shape= (109, 432)
first_test_shape= (105, 432)
classes_quantity= 2
tr_lbls=	 {1, 2}
Count_labels= [52 57]
max(train_feature_Altitude)= 8.0329
min(train_feature_Altitude)= -2.0538
first_train_sample=
 [ 1.         0.11403    0.10489    0.095754   0.086619   0.086619
  0.086619   0.077484   0.077484   0.077484   0.077484   0.077484
  0.077484   0.077484   0.077484   0.077484   0.077484   0.077484
  0.068348   0.068348   0.068348   0.068348   0.068348   0.068348
  0.068348   0.068348   0.068348   0.077484   0.077484   0.077484
  0.077484   0.086619   0.086619   0.095754   0.095754   0.10489
  0.11403    0.11403    0.12316    0.12316    0.1323     0.14143
  0.14143    0.15057    0.1597     0.1597     0.16884    0.17797
  0.19624    0.20538    0.22365    0.25106    0.30587    0.36068
  0.37895    0.37895    0.43376    0.49771    0.47031    0.45204
  0.70783    1.2742     1.5026     1.4843     1.3564     1.2742
  1.192      1.0184     0.91794    0.89054    0.85399    0.70783
  0.

In [4]:
np.shape(ecg)

(109, 481)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

each class and its windows =
1 52	2 57	

<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
rpt_cnn_clsfctn=3                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
aug_amnt_Ttl=np.empty((0,5), int) 
rslts=np.array([])
for X in range(1,5):                               #داده افزایی تا چند برابر کلاس اکثریت
    mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
    mx_wndws=int(X*mx_wndws)          #
    for alpha in range(1,4):
        rprt=np.array([])
        for beta in range(1,4):                        #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
            if beta==3:
                beta=beta*2
            scors0=np.array([78,76])    #76,72 - 82,74 - 75,69     #Class Recalls without Augmentation
            AgScr=1+alpha*((100-scors0)/(np.max(100-scors0)))**beta        #Class Specific

            f_scr=np.array([69,67,72,75,74,70])                      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
            f_scr_difrnc=f_scr-f_scr[0]
            for z in range (len(f_scr_difrnc)):
                if f_scr_difrnc[z]<0:
                    f_scr_difrnc[z]=0
            f_scr_difrnc=f_scr_difrnc**beta
            f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
            aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

            for i in range (1,cls_num+1):
                aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*AgScr[i-1]-cls_wndws[i-1])],axis=0)
                #vlum_win=mx_wndws-cls_wndws[i-1]                        #میزان داده افزایی در روش های عمیق (پنجره)
            aug_amnt=np.int16(np.round(aug_amnt))
            aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
            #print(np.int16(100*f_scr_rtio))
            #print(aug_amnt)
            rprt=np.empty((0,3),float)
            print(aug_amnt)
            for repeat in range(1,rpt_cnn_clsfctn+1):
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num+1):                            #تعریف آرایه ی پنجره های کلاس ها
                    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                    if aug_amnt[i-1,0]>0:
                        methd=10
                        sort=-5  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                        dp_slct_dstrb=1            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,1]>0:
                        methd=20
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,2]>0:
                        methd=30
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,3]>0:
                        methd=40
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,4]>0:
                        methd=50
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort),axis=0)
                    ''''if aug_amnt[i-1,5]>0:
                        methd=60
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i-1,6]>0:
                        methd=70
                        sort=-1
                        dp_slct_dstrb=2
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort),axis=0)'''
                    #print('class ', i, 'train shape = ', np.shape(xtrain))

                # Normalization train windows
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


                ###Test Windows:
                wndws_test=np.empty((0,len(wndws1[0])), float)
                i=0
                windws=np.array([])
                for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

                # Normalization test windows
                for i in range (1,cls_num+1):                
                    cls=i
                    #print('\n cls', i, ' >> ')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    #print('after normalizing >>')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print("each class and its windows =")                    #تجمیع کلاس های تست
                for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(Xtrain)
                y_train=np.array(ytrain)
                X_test=np.array(Xtest)
                y_test=np.array(ytest)
                X_valid=np.array(Xtrain)
                y_valid=np.array(ytrain)
                #print('X_train => ', X_train.shape)
                #print('y_train => ', y_train.shape)
                #print('X_test  => ', X_test.shape)
                #print('y_test  => ', y_test.shape)
                #print('X_valid  => ', X_valid.shape)
                #print('y_valid  => ', y_valid.shape)

                if min(y_train)==1:
                    y_train=y_train-1
                    y_test=y_test-1
                    y_valid =y_valid-1
                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                ####### NETWORK #######
                accuracy=0
                acc_crs=np.array([])
                ##for i in range (10):
                ##    if accuracy<0.2 :
                verbose, epochs, batch_size = 0, 1000, btch
                n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
                steps_per_epoch = len(X_train)//batch_size
                validation_steps = len(X_valid)//batch_size # if you have test data
                model_crs = Sequential()
                BatchNormalization()
                model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
                #model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
                #model_crs.add(Dropout(0.5))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''
                model_crs.add(Flatten())
                #model_crs.add(Dropout(0.5))
                model_crs.add(Dense(2000, activation='relu'))
                model_crs.add(Dense(n_outputs, activation='softmax'))
                model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy
                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)
                model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
                # evaluate model_crs
                _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
                predict_x=model_crs.predict(X_test)              # Function 1
                y_pred_crs=np.argmax(predict_x,axis=1)           # function from augment.py to remove 0 index predictions
                rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
                rprt_row=np.array([])
                rprt_row=np.append(rprt_row,rprt0['accuracy'])
                rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
                rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
                rprt=np.append(rprt,[rprt_row],axis=0)
            #print('rprt=',rprt)
            maxm=rprt[np.argmax(rprt[:,1])]
            #print('maxm=',maxm)
            avrg=np.mean(rprt,axis=0)
            #print('avrg=',avrg)
            rslts=np.append(rslts,['X '+str(X)+'  alpha '+str(alpha) +'  Beta '+str(beta)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
            print('rslts=',rslts[-1])


 max = 58
[[ 0 11 23 19  4]
 [ 0 11 23 19  4]]
4/4 [==============================] - 0s 3ms/step
rslts= X 1  alpha 1  Beta 1  average= [74 74 74]  max= [77 77 77]  var= [2 2 2]
[[ 0  7 27 19  1]
 [ 0  7 29 20  1]]
4/4 [==============================] - 0s 2ms/step
rslts= X 1  alpha 1  Beta 2  average= [74 74 74]  max= [79 79 79]  var= [12 12 12]
[[ 0  0 29 10  0]
 [ 0  1 42 14  0]]
4/4 [==============================] - 0s 2ms/step
rslts= X 1  alpha 1  Beta 6  average= [73 73 73]  max= [76 76 76]  var= [2 2 2]
[[ 0 22 44 36  7]
 [ 0 23 46 38  8]]
4/4 [==============================] - 0s 2ms/step
rslts= X 1  alpha 2  Beta 1  average= [79 79 79]  max= [81 81 81]  var= [5 5 5]
[[ 0 13 51 35  1]
 [ 0 14 58 40  2]]
4/4 [==============================] - 0s 3ms/step
rslts= X 1  alpha 2  Beta 2  average= [78 78 78]  max= [80 80 80]  var= [12 12 12]
[[ 0  1 54 18  0]
 [ 0  1 84 28  0]]
4/4 [==============================] - 0s 2ms/step
rslts= X 1  alpha 2  Beta 6  average= [77 77 77]  max= 

4/4 [==============================] - 0s 3ms/step
rslts= X 4  alpha 1  Beta 6  average= [75 75 75]  max= [76 75 75]  var= [0 0 0]
[[  0 119 238 198  40]
 [  0 125 251 209  42]]
4/4 [==============================] - 0s 3ms/step
rslts= X 4  alpha 2  Beta 1  average= [77 77 77]  max= [78 78 78]  var= [0 0 0]
[[  0  71 284 197   8]
 [  0  79 318 221   9]]
4/4 [==============================] - 0s 3ms/step
rslts= X 4  alpha 2  Beta 2  average= [79 79 79]  max= [80 80 80]  var= [1 1 1]
[[  0   5 331 111   0]
 [  0   7 464 155   0]]
4/4 [==============================] - 0s 3ms/step
rslts= X 4  alpha 2  Beta 6  average= [76 76 76]  max= [78 78 78]  var= [2 3 3]
[[  0 161 321 268  54]
 [  0 171 342 285  57]]
4/4 [==============================] - 0s 2ms/step
rslts= X 4  alpha 3  Beta 1  average= [77 77 77]  max= [80 80 80]  var= [2 2 2]
[[  0  95 381 264  11]
 [  0 108 434 301  12]]
4/4 [==============================] - 0s 3ms/step
rslts= X 4  alpha 3  Beta 2  average= [76 76 76]  max= [78 

In [9]:
rslts

array(['X 1  alpha 1  Beta 1  average= [74 74 74]  max= [77 77 77]  var= [2 2 2]',
       'X 1  alpha 1  Beta 2  average= [74 74 74]  max= [79 79 79]  var= [12 12 12]',
       'X 1  alpha 1  Beta 6  average= [73 73 73]  max= [76 76 76]  var= [2 2 2]',
       'X 1  alpha 2  Beta 1  average= [79 79 79]  max= [81 81 81]  var= [5 5 5]',
       'X 1  alpha 2  Beta 2  average= [78 78 78]  max= [80 80 80]  var= [12 12 12]',
       'X 1  alpha 2  Beta 6  average= [77 77 77]  max= [80 79 79]  var= [5 5 5]',
       'X 1  alpha 3  Beta 1  average= [75 75 75]  max= [75 75 75]  var= [0 0 0]',
       'X 1  alpha 3  Beta 2  average= [78 78 78]  max= [80 80 80]  var= [6 6 6]',
       'X 1  alpha 3  Beta 6  average= [72 72 72]  max= [74 74 74]  var= [4 4 4]',
       'X 2  alpha 1  Beta 1  average= [73 72 72]  max= [73 73 73]  var= [0 0 0]',
       'X 2  alpha 1  Beta 2  average= [78 78 78]  max= [81 81 81]  var= [5 5 5]',
       'X 2  alpha 1  Beta 6  average= [72 72 72]  max= [76 76 76]  var= [ 9 10 1